In [1]:
import json
import random

In [2]:
with open('shapenet/partnet_metadata.json', 'r') as f:
    partnet_metadata = json.load(f)
metadata_zip = list(zip(partnet_metadata['model_ids'], partnet_metadata['category_ids'], partnet_metadata['captions']))
metadata_zip = sorted(metadata_zip, key=lambda x: str(x[1]) + str(x[0]))
with open('shapenet/partnet_metadata_sorted_zip.json', 'w') as f:
    json.dump(metadata_zip, f)

In [3]:
print(partnet_metadata.keys())
# with open('shapenet/binvox_metadata.json', 'r') as f:
#     binvox_metadata = json.load(f)
# print(binvox_metadata["model_ids"][:20])
# print(partnet_metadata["model_ids"][:20])
# issue = 0
# for model_id in partnet_metadata["model_ids"]:
#     if model_id not in binvox_metadata["model_ids"]:
#         issue += 1
# print(issue)

dict_keys(['model_ids', 'captions', 'category_ids'])


In [4]:
print(len(metadata_zip))

13919


In [5]:
current_cat_id = ""
cat_ids = []
indexes = []
for i, model_data in enumerate(metadata_zip):
    _, cat_id, _ = model_data
    if cat_id != current_cat_id:
        cat_ids.append(cat_id)
        indexes.append(i)
        current_cat_id = cat_id
indexes.append(len(metadata_zip))
print(indexes)
print(len(indexes))

[0, 204, 668, 809, 5558, 6092, 6606, 7160, 8374, 8805, 9014, 13919]
12


In [6]:
print("cat_ids", cat_ids)
print("indexes", indexes)
cat_id_suffled_ranges = []
for i in range(len(indexes) - 1):
    cat_range = [j for j in range(indexes[i], indexes[i + 1])]
    random.shuffle(cat_range)
    cat_id_suffled_ranges.append(cat_range)

cat_ids [2818832, 2876657, 2880940, 3001627, 3046257, 3325088, 3593526, 3636649, 3642806, 3797390, 4379243]
indexes [0, 204, 668, 809, 5558, 6092, 6606, 7160, 8374, 8805, 9014, 13919]


In [7]:
print(cat_id_suffled_ranges[0][0:20])
train_indexes = []
test_indexes = []
total_num = 0
for cat_range in cat_id_suffled_ranges:
    total_num += len(cat_range)
print(total_num)
for cat_range in cat_id_suffled_ranges:
    train_number = int(len(cat_range) * 0.9)
    train_indexes.extend(cat_range[:train_number])
    test_indexes.extend(cat_range[train_number:])
print(len(train_indexes), len(test_indexes), len(train_indexes)+len(test_indexes))

[201, 128, 150, 17, 61, 88, 122, 66, 60, 127, 100, 98, 139, 57, 82, 144, 89, 1, 5, 130]
13919
12521 1398 13919


In [8]:
with open('shapenet/train_indexes.json', 'w') as f:
    json.dump(train_indexes, f)
with open('shapenet/test_indexes.json', 'w') as f:
    json.dump(test_indexes, f)

In [9]:
with open('shapenet/train_indexes.json', 'r') as f:
    indexes = json.load(f)
    print(len(indexes))
    print(indexes[0:20])
with open('shapenet/test_indexes.json', 'r') as f:
    indexes = json.load(f)
    print(len(indexes))
    print(indexes[0:20])

12521
[201, 128, 150, 17, 61, 88, 122, 66, 60, 127, 100, 98, 139, 57, 82, 144, 89, 1, 5, 130]
1398
[103, 31, 26, 20, 75, 92, 199, 181, 118, 32, 135, 180, 102, 95, 2, 19, 119, 8, 184, 137]


In [10]:
import h5py
import numpy as np

model_ids, categories, captions = zip(*metadata_zip)
print(type(model_ids[0]), type(categories[0]), type(captions[0]))
with h5py.File('shapenet/binvox.hdf5','w') as hf:
    np_captions = np.array([str(i).strip() for i in captions], dtype=h5py.special_dtype(vlen=str)) 
    hf.create_dataset('captions', data=np_captions)
    np_model_ids = np.array(model_ids, dtype=h5py.special_dtype(vlen=str))
    hf.create_dataset('model_ids', data=np_model_ids)
    np_category_ids = np.array(categories)
    hf.create_dataset('category_ids', data=np_category_ids)
with h5py.File('shapenet/binvox.hdf5','r') as hf:
    print(list(i.decode() for i in hf['captions'][:50]))

<class 'str'> <class 'int'> <class 'str'>
['bunk bed that is short and average length and skinny.  it has two posts . it has two beds .', 'platform bed that is very thick and short and skinny.  it has four posts . it has two beds .', 'bunk that is short and average length and wide.  it has a ladder.  it has four posts . it has two beds .', 'platform bed that is thin and long and average width.  it has a headboard.  it has two pillows . it has one bed .', 'bunk bed that is short and very short and wide.  it has a ladder.  it has four posts . it has two beds .', 'bunk bed that is tall and short and wide.  it has a ladder.  it has many posts . it has two beds .', 'bunk bed that is tall and short and average width.  it has a headboard.  it has three pillows . it has one bed .', 'bunk bed that is average height and average length and average width.  it has a ladder.  it has four pillows . it has many posts . it has two beds .', 'platform bed that is average thickness and average length and 